# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
import os.path
import pickle
has_train_data = os.path.isfile('train.p') 
if not has_train_data:
    import urllib.request as request
    train_url = "https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580ad86d_train/train.p"
    test_url = "https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580ad89e_test/test.p"
    place_holder = request.urlretrieve(train_url,'train.p')
    place_holder = request.urlretrieve(test_url,'test.p')
    
def load_train_test_data():
    train_data = 'train.p'
    test_data = 'test.p'
    with open(train_data, 'rb') as f:
        pickle_data = pickle.load(f)
        X_train, y_train = pickle_data['features'],pickle_data['labels']   
    del pickle_data  # Free up memory
    return X_train, y_train


from sklearn.utils import shuffle
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
## This method will load the train data, normailze it, shuffle it and split into train/validation data
def reload_train_validation_data():
    X_train, y_train = load_train_test_data()
    X_train = X_train.astype('float32')
    X_train /= 255 ## => sets the values between 0 and 1
    X_train -= 0.5 ## => sets the values between -0.5 and 0.5
    Y_train = np_utils.to_categorical(y_train, 43)
    X_train, Y_train = shuffle(X_train, Y_train, random_state=123)
    return train_test_split(X_train,Y_train,test_size=0.25,random_state=123)
X_train, X_val, Y_train, Y_val = reload_train_validation_data()

Using TensorFlow backend.


In [2]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

In [3]:
n_inputs = 32*32*3
X_train = X_train.reshape(len(X_train), n_inputs)
X_val = X_val.reshape(len(X_val), n_inputs)

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [4]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers.core import Dense, Activation

n_layer = 128
n_classes = 43

model = Sequential()
model.add(Dense(n_layer, input_shape=(n_inputs,),name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_layer,name="hidden2"))
model.add(Activation('relu'))
model.add(Dense(n_classes,name="output"))
model.add(Activation('softmax'))

model.summary()



# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
hidden1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           hidden1[0][0]                    
____________________________________________________________________________________________________
hidden2 (Dense)                  (None, 128)           16512       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 128)           0           hidden2[0][0]                    
___________________________________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [5]:
# TODO: Compile and train the model here.
n_epoch = 2
batch_size = 100

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=n_epoch, verbose=1)

assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/2
29406/29406 [==============================] - 6s - loss: 1.7880 - acc: 0.5165     
Epoch 2/2
29406/29406 [==============================] - 5s - loss: 0.7231 - acc: 0.7943     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [6]:

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=n_epoch, verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 6s - loss: 0.5128 - acc: 0.8524 - val_loss: 0.7796 - val_acc: 0.8032
Epoch 2/2
29406/29406 [==============================] - 6s - loss: 0.4196 - acc: 0.8841 - val_loss: 0.4396 - val_acc: 0.8729


**Validation Accuracy**:

In [7]:
score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation Accuracy:', score[1])

Validation Accuracy: 0.872896052253


## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

##reload the data
X_train, X_val, Y_train, Y_val = reload_train_validation_data()


##set parameters
input_shape = X_train[0].shape
kernel_size = (3, 3)
n_filters = 32
n_layer = 1024
batch_size = 500
n_epoch = 5
n_classes = 43
pool_size = (2, 2)

##create the model
model = Sequential()
model.add(Convolution2D(n_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(n_layer))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

##model.summary() => visulize the model

history = model.fit(X_train, Y_train, batch_size=batch_size, 
                    nb_epoch=n_epoch,verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/5
29406/29406 [==============================] - 230s - loss: 1.7949 - acc: 0.5538 - val_loss: 0.7309 - val_acc: 0.8093
Epoch 2/5
29406/29406 [==============================] - 232s - loss: 0.4825 - acc: 0.8789 - val_loss: 0.3890 - val_acc: 0.8971
Epoch 3/5
29406/29406 [==============================] - 239s - loss: 0.2601 - acc: 0.9411 - val_loss: 0.2382 - val_acc: 0.9504
Epoch 4/5
29406/29406 [==============================] - 231s - loss: 0.1687 - acc: 0.9620 - val_loss: 0.1939 - val_acc: 0.9519
Epoch 5/5
29406/29406 [==============================] - 231s - loss: 0.1294 - acc: 0.9701 - val_loss: 0.1692 - val_acc: 0.9597


**Validation Accuracy**: (fill in here)

In [9]:
score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation Accuracy:', score[1])

Validation Accuracy: 0.959706212414


## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [11]:
##reload the data
X_train, X_val, Y_train, Y_val = reload_train_validation_data()


##set parameters
input_shape = X_train[0].shape
kernel_size = (3, 3)
n_filters = 32
n_layer = 1024
batch_size = 500
n_epoch = 5
n_classes = 43
pool_size = (2, 2)

##create the model
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

model = Sequential()
model.add(Convolution2D(n_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(n_layer))
model.add(Activation('relu'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#model.summary()

history = model.fit(X_train, Y_train, batch_size=batch_size, 
                    nb_epoch=n_epoch,verbose=1, validation_data=(X_val, Y_val))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/5
29406/29406 [==============================] - 82s - loss: 1.9136 - acc: 0.4939 - val_loss: 0.9673 - val_acc: 0.7537
Epoch 2/5
29406/29406 [==============================] - 82s - loss: 0.6823 - acc: 0.8231 - val_loss: 0.4936 - val_acc: 0.8711
Epoch 3/5
29406/29406 [==============================] - 83s - loss: 0.3843 - acc: 0.9061 - val_loss: 0.3286 - val_acc: 0.9224
Epoch 4/5
29406/29406 [==============================] - 90s - loss: 0.2612 - acc: 0.9362 - val_loss: 0.2650 - val_acc: 0.9278
Epoch 5/5
29406/29406 [==============================] - 108s - loss: 0.1926 - acc: 0.9555 - val_loss: 0.2254 - val_acc: 0.9399


**Validation Accuracy**:

In [12]:
score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation Accuracy:', score[1])

Validation Accuracy: 0.939916352168


## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [13]:
from keras.layers.core import Dropout

X_train, X_val, Y_train, Y_val = reload_train_validation_data()


##set parameters
input_shape = X_train[0].shape
kernel_size = (3, 3)
n_filters = 32
n_layer = 1024
batch_size = 500
n_epoch = 5
pool_size = (2, 2)

##create the model
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

model = Sequential()
model.add(Convolution2D(n_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25)) ## => keep 75% of the input units
model.add(Flatten())
model.add(Dense(n_layer))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

history = model.fit(X_train, Y_train, batch_size=batch_size, 
                    nb_epoch=n_epoch,verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 32, 32, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_10 (Activation)       (None, 32, 32, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 16, 16, 32)    0           activation_10[0][0]              
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 16, 16, 32)    0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [ ]:
**Validation Accuracy**:

In [15]:
score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation Accuracy:', score[1])

Validation Accuracy: 0.959604202789


## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [16]:
from keras.layers.core import Dropout

X_train, X_val, Y_train, Y_val = reload_train_validation_data()

##set parameters
input_shape = X_train[0].shape
kernel_size = (3, 3)
n_filters = 32
n_layer_1 = 1024
n_layer_2 = 512
batch_size = 500
n_epoch = 10
pool_size = (2, 2)

##create the model
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

model = Sequential()
##First conv-net layer
model.add(Convolution2D(n_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
##Second conv-net layer
model.add(Convolution2D(n_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=pool_size)) # => pooling
model.add(Dropout(0.25)) 
model.add(Flatten())
##third layer
model.add(Dense(n_layer_1)) 
model.add(Activation('relu'))
model.add(Dropout(0.25))

##fourth layer
model.add(Dense(n_layer_2)) 
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

#model.summary()

history = model.fit(X_train, Y_train, batch_size=batch_size, 
                    nb_epoch=n_epoch,verbose=1, validation_data=(X_val, Y_val))

Train on 29406 samples, validate on 9803 samples
Epoch 1/10
29406/29406 [==============================] - 182s - loss: 3.1608 - acc: 0.1664 - val_loss: 2.2274 - val_acc: 0.3690
Epoch 2/10
29406/29406 [==============================] - 188s - loss: 2.0568 - acc: 0.3933 - val_loss: 1.4997 - val_acc: 0.5240
Epoch 3/10
29406/29406 [==============================] - 192s - loss: 1.3871 - acc: 0.5666 - val_loss: 0.9620 - val_acc: 0.6968
Epoch 4/10
29406/29406 [==============================] - 221s - loss: 0.9112 - acc: 0.7089 - val_loss: 0.8309 - val_acc: 0.7286
Epoch 5/10
29406/29406 [==============================] - 228s - loss: 0.5907 - acc: 0.8117 - val_loss: 0.3013 - val_acc: 0.9129
Epoch 6/10
29406/29406 [==============================] - 230s - loss: 0.3950 - acc: 0.8764 - val_loss: 0.2380 - val_acc: 0.9344
Epoch 7/10
29406/29406 [==============================] - 224s - loss: 0.2910 - acc: 0.9128 - val_loss: 0.2240 - val_acc: 0.9302
Epoch 8/10
29406/29406 [========================

**Best Validation Accuracy:**

In [18]:
score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation Accuracy:', score[1])

Validation Accuracy: 0.981740283587


## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [19]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

score = model.evaluate(X_test, Y_test)

12630/12630 [==============================] - 28s    


**Test Accuracy:**

In [20]:
print('Test Accuracy:', score[1])

Test Accuracy: 0.882660332504


## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.